# Machine Learning
## Gaussian Naive Bayes Classifier

In [1]:
import numpy as np
import pandas as pd
import math

# Importing Data
df = pd.read_csv("/Users/harshitsrivastava/Desktop/Data/glasshw1.csv", header = None)
df = df.drop([0], axis=1)

In [2]:
df.head()

,1,2,3,4,5,6,7,8,9,10
0,1.51824,12.87,3.48,1.29,72.95,0.60,8.43,0.00,0.00,1
1,1.51832,13.33,3.34,1.54,72.14,0.56,8.99,0.00,0.00,1
2,1.51747,12.84,3.50,1.14,73.27,0.56,8.55,0.00,0.00,1
3,1.51775,12.85,3.48,1.23,72.97,0.61,8.56,0.09,0.22,1
4,1.51768,12.65,3.56,1.30,73.08,0.61,8.69,0.00,0.14,1


## Problems
1. What was the estimated value of P(C) for C = 1?
2. What was the estimated value of P(C) for C = 2?
3. What were the estimated values for (ˆµ, ˆσ^2) for the Gaussians corresponding to attribute Refractive Index and Class 1.
4. What were the estimated values for (ˆµ, ˆσ^2) for the Gaussians corresponding to attribute Calcium and Class 2.
5.  Which classes were predicted for the following examples: Examples 20, 60, 100, 140, and 180
6. What was the percentage training error?

In [3]:
### 1. What was the estimated value of P(C) for C = 1?
### 2. What was the estimated value of P(C) for C = 2?

# Storing the count of different classes in 'c'
c = df.groupby(10).size()

pC1 = float(c[1])/200
pC2 = float(c[2])/200

print "Probability of C = 1, P(C=1):", pC1
print "Probability of C = 2, P(C=2):", pC2

Probability of C = 1, P(C=1): 0.745
Probability of C = 2, P(C=2): 0.255


In [4]:
### 3. What were the estimated values for (µ, σ^2) for the Gaussians corresponding to attribute Refractive Index and Class 1.
### 4. What were the estimated values for (µ, σ^2) for the Gaussians corresponding to attribute Calcium and Class 2.

# Dividing the data into two subsets based on the Classes
C1 = df[(df[10]==1)]
C2 = df[(df[10]==2)]

print "Refractive Index"
print "Class 1- Mean:", C1[1].mean(), ", Variance:", C1[1].var()

print "\n\nCalcium"
print "Class 2- Mean:", C2[7].mean(), ", Variance:", C2[7].var()

Refractive Index
Class 1- Mean: 1.51866181208 , Variance: 9.94025007256e-06


Calcium
Class 2- Mean: 9.06019607843 , Variance: 2.50927396078


### Prediction

In [5]:
# Calculating Mean and Variance for each feature classwise and appending it into a dictionary

def Summarize(C1, C2):
    Class = {1: [], 2: []}
    for x in range(1, 10):
        Class[1].append([C1[x].mean(), C1[x].var()])
        Class[2].append([C2[x].mean(), C2[x].var()])
    
    return Class

Class = Summarize(C1, C2)
Class

{1: [[1.518661812080537, 9.940250072555836e-06],
  [13.198859060402686, 0.36471017594775984],
  [3.2718120805369137, 0.8551027752584802],
  [1.276308724832215, 0.10965587701795754],
  [72.57845637583893, 0.4273401686921824],
  [0.4704026845637584, 0.05063902593869039],
  [8.968724832214763, 2.0139639035008168],
  [0.031879194630872486, 0.07023022855069852],
  [0.07087248322147648, 0.010429639035008181]],
 2: [[1.5176378431372552, 8.462929254902018e-06],
  [14.066666666666666, 1.1381386666666666],
  [0.7337254901960785, 1.2131758431372552],
  [1.9666666666666672, 0.35775866666666656],
  [72.85549019607845, 1.1676652549019586],
  [0.5596078431372549, 1.6517558431372548],
  [9.06019607843137, 2.5092739607843138],
  [0.63921568627451, 0.5564113725490198],
  [0.023137254901960787, 0.006825960784313725]]}

In [6]:
def prob_distr_func(mean, var, x):
    # Function to obtain Probability of Gaussian Distribution 
    
    denom = (2*(math.pi)*var)**.5
    num = math.exp(-(float(x)-float(mean))**2/(2*var))
    return num/denom

In [7]:
df[11] = 0
df[12] = 0
df[13] = 0

def prob(data):
    Probabilities = {}
    for j in range(0, len(data)):
        
        # Initializing probabilities for each class with the priors
        Probabilities[1] = math.log1p(pC1)
        Probabilities[2] = math.log1p(pC2)
        
        for i in range(0, 9):
            prob1 = prob_distr_func(Class[1][i][0], Class[1][i][1], data.iloc[j, i])
            prob2 = prob_distr_func(Class[2][i][0], Class[2][i][1], data.iloc[j, i])
            
            # Smoothing the '0' values obtained from the Probability Distribution Function
            if prob1 == 0:
                prob1 = 0.1
                
            if prob2 == 0:
                prob2 = 0.1
                
            # Taking summation of Log Likelihoods
            Probabilities[1] += np.log(prob1)
            Probabilities[2] += np.log(prob2)
           
        # Storing the Probabilities for each class
        df.iloc[j, 10] = Probabilities[1]
        df.iloc[j, 11] = Probabilities[2]
        
        # Classifying into Classes based on probabilities obtained
        if Probabilities[1] > Probabilities[2]:
            df.iloc[j, 12] = 1
        else:
            df.iloc[j, 12] = 2
            

In [8]:
# Calling the above functions
prob(df)

# Counting the incorrect Predictions
count = 0
for _, x in df.iterrows():
    if x[10] != x[13]:
        count += 1
        
print "Training Error:", (count/float(len(df)))*100

Training Error: 10.0


In [9]:
# Original DataFrame containing Probabilites of Class A and B at 11th & 12th columns respectively
# Column 13 contains Predicted class for that example 
df

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.51824,12.87,3.48,1.29,72.95,0.60,8.43,0.00,0.00,1,4.078802,-4.741227,1
1,1.51832,13.33,3.34,1.54,72.14,0.56,8.99,0.00,0.00,1,4.008089,-3.798995,1
2,1.51747,12.84,3.50,1.14,73.27,0.56,8.55,0.00,0.00,1,3.616117,-5.155989,1
3,1.51775,12.85,3.48,1.23,72.97,0.61,8.56,0.09,0.22,1,3.162409,-7.536513,1
4,1.51768,12.65,3.56,1.30,73.08,0.61,8.69,0.00,0.14,1,3.654710,-6.064803,1
5,1.51769,12.45,2.71,1.29,73.70,0.56,9.06,0.00,0.24,1,0.976333,-7.367986,1
6,1.51892,13.46,3.83,1.26,72.55,0.57,8.21,0.00,0.14,1,4.151280,-6.312023,1
7,1.51727,14.70,0.00,2.34,73.28,0.00,8.95,0.66,0.00,2,-15.505971,-0.667420,2
8,1.51545,14.14,0.00,2.68,73.39,0.08,9.07,0.61,0.05,2,-16.745535,-1.316368,2
9,1.51994,13.27,0.00,1.76,73.03,0.47,11.32,0.00,0.00,2,-4.364716,-2.165500,2


In [10]:
print "20th Example label:", df[13][19]
print "60th Example label:", df[13][59]
print "100th Example label:", df[13][99]
print "140th Example label:", df[13][139]
print "180th Example label:", df[13][179]

20th Example label: 1
60th Example label: 1
100th Example label: 1
140th Example label: 2
180th Example label: 2


## Cross-Validation

In [11]:
def cv_label(data, Class):
    Probabilities = {}
    labels = []
    for j in range(0, len(data)):
        
        # Initializing probabilities for each class with the priors
        Probabilities[1] = math.log1p(pC1)
        Probabilities[2] = math.log1p(pC2)
        
        for i in range(0, 9):
            prob1 = prob_distr_func(Class[1][i][0], Class[1][i][1], data.iloc[j, i])
            prob2 = prob_distr_func(Class[2][i][0], Class[2][i][1], data.iloc[j, i])
            
            # Smoothing the '0' values obtained from the Probability Distribution Function
            if prob1 == 0:
                prob1 = 0.1
                
            if prob2 == 0:
                prob2 = 0.1
                
            # Taking summation of Log Likelihoods
            Probabilities[1] += np.log(prob1)
            Probabilities[2] += np.log(prob2)
            
        # Classifying into Classes based on probabilities obtained
        if Probabilities[1] > Probabilities[2]:
            labels.append(1)
        else:
            labels.append(2)
    
    return labels

In [12]:
df = df.drop([11, 12, 13], axis = 1)
Folds = {}
Folds[1], Folds[2], Folds[3], Folds[4], Folds[5] = np.array_split(df, 5)
train = pd.DataFrame()
Master_test = pd.DataFrame()
score = []
output = pd.DataFrame()

# Loop for different folds
for i in range(1, 6):
    test = Folds[i]
    j = 1
    # Loop to combine the training sets
    while j < 6:
        if i == j:
            j += 1
        else:
            train = train.append(Folds[j])
            j += 1
    C1 = train[(train[10]==1)]
    C2 = train[(train[10]==2)]
    Class_cv = Summarize(C1, C2)
    test[11] = cv_label(test, Class_cv)
    
    # Appending  results of test set into a Master_test set
    Master_test = Master_test.append(test)
    
    # Counting the incorrect predictions
    count = 0
    for _, t in test.iterrows():
        output = output.append([t[11]])
        if t[10] == t[11]:
            count += 1

    print "Fold:", i, "; Training Error:", (count/float(len(test)))*100
    score.append((count/float(len(test)))*100)
        
# Extracting the output dataframe as CSV file
output.to_csv("CV_Output.csv")
# Printing Error rate
print "Avg Cross Validation Error:", np.mean(score)

Fold: 1 ; Training Error: 92.5
Fold: 2 ; Training Error: 87.5
Fold: 3 ; Training Error: 95.0
Fold: 4 ; Training Error: 85.0
Fold: 5 ; Training Error: 90.0
Avg Cross Validation Error: 90.0


In [13]:
print "20th Example label:", Master_test[11][19]
print "60th Example label:", Master_test[11][59]
print "100th Example label:", Master_test[11][99]
print "140th Example label:", Master_test[11][139]
print "180th Example label:", Master_test[11][179]

20th Example label: 1
60th Example label: 1
100th Example label: 1
140th Example label: 2
180th Example label: 2


## Zero - R Method

In [14]:
#df = df.drop([11, 12, 13], axis = 1)
Folds = {}
Folds[1], Folds[2], Folds[3], Folds[4], Folds[5] = np.array_split(df, 5)
train = pd.DataFrame()
score = []

# Loop for different folds
for i in range(1, 6):
    test = Folds[i]
    j = 1
    # Loop to combine the training sets
    while j < 6:
        if i == j:
            j += 1
        else:
            train = train.append(Folds[j])
            j += 1
            
    # Getting the majority class in the test set and setting it as 'Predicted Labels'
    test[11] = int(test[10].mode())
    
    # Counting the incorrect predictions
    count = 0
    for _, t in test.iterrows():
        if t[10] == t[11]:
            count += 1

    print "Fold:", i, ", Training Error:", (count/float(len(test)))*100
    score.append((count/float(len(test)))*100)
        
print "Avg Cross Validation Error:", np.mean(score)

Fold: 1 , Training Error: 72.5
Fold: 2 , Training Error: 82.5
Fold: 3 , Training Error: 75.0
Fold: 4 , Training Error: 72.5
Fold: 5 , Training Error: 70.0
Avg Cross Validation Error: 74.5
